In [ ]:
import torch
# If there"s a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
# If not...
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

In [ ]:
!nvidia-smi

In [3]:
%%capture
!pip install "sagemaker==2.69.0" "transformers==4.12.3" --upgrade

In [1]:
import sagemaker
# assert sagemaker.__version__ >= "2.69.0"

In [2]:
sagemaker.__version__ 

'2.68.0'

In [ ]:
from sagemaker import get_execution_role
import sagemaker.huggingface

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [15]:
print(f"sagemaker role arn: {role}")
# print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::197614225699:role/bi-sagemaker-access
sagemaker session region: us-east-1


In [16]:
bucket='emotion-class-197614225699'

In [17]:
training_input_path = f's3://{bucket}/processing_output/train_data'
val_input_path = f's3://{bucket}/processing_output/validation_data'

In [18]:
val_input_path

's3://emotion-class-197614225699/processing_output/validation_data'

In [20]:
# from huggingface_hub import notebook_login

# notebook_login()

### Set up Huggingface training job

In [44]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 10,
                 'train_batch_size': 16,
                 'learning_rate': 3e-5, 
                 'fp16': True,
                 'model_name':'roberta-base',
                 'weight_decay':0.01,
                 'do_eval': True,
                 'load_best_model_at_end':True
                 }

# #configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True}}}


In [5]:
#### check training script

In [4]:
!pygmentize ./scripts/train.py

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,roc_auc_score
from datasets import load_from_disk
import random
import logging
import sys
import argparse
import os
import torch

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=5)
    parser.add_argument("--train_batch_size", type=int, default=16)
    parser.add_argument("--eval_batch_size", type=int, default=32)
    parser.add_argument("--warmup_steps", type=int, default=500)
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    
        
#     # Push to Hub Parameters
#     parser.add_argument("--push_to_hub", type=bool

In [46]:
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.16xlarge',
    instance_count=1,
    volume_size=1000,
    role=role,
    transformers_version = '4.12',           # the transformers version used in the training job
    pytorch_version      = '1.9',           # the pytorch_version version used in the training job
    py_version           = 'py38',            # the python version used in the training job
    output_path=f's3://{bucket}/training_output/',
    base_job_name="emotion-training",
    hyperparameters=hyperparameters,
#     disable_profiler=True,
#     distribution=distribution,
)

In [47]:
import time

start = time.time()


# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'val': val_input_path})

end = time.time()

print(end - start)

2021-12-03 20:58:03 Starting - Starting the training job...
2021-12-03 20:58:11 Starting - Launching requested ML instancesProfilerReport-1638565083: InProgress
.........
2021-12-03 20:59:54 Starting - Preparing the instances for training.........
2021-12-03 21:01:30 Downloading - Downloading input data...
2021-12-03 21:01:50 Training - Downloading the training image...........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-03 21:06:25,475 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-03 21:06:25,550 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-03 21:06:26,968 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-03 21:06:27,756 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs

Using amp fp16 backend
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
***** Running training *****
Num examples = 16000
  Num Epochs = 10
Num examples = 16000
  Num Epochs = 10
Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
0%|          | 0/1250 [00:00<?, ?it/s]
[2021-12-03 21:06:49.000 algo-1:26 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-12-03 21:06:


2021-12-03 21:06:52 Training - Training image download completed. Training in progress.algo-1:26:26 [0] ofi_init:1134 NCCL WARN NET/OFI Only EFA provider is supported
NCCL version 2.7.8+cuda11.1
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
10%|█         | 125/1250 [01:23<09:27,  1.98it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
***** Running Evaluation *****
Num examples = 2000
Batch size = 256
Num examples = 2000
  Batch size = 256
62%|██████▎   | 5/8 [00:01<00:0

100%|██████████| 8/8 [00:02<00:00,  2.93it/s]#033[A
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
#033[A
{'eval_loss': 1.043243408203125, 'eval_accuracy': 0.6115, 'eval_f1': 0.515432386199899, 'eval_precision': 0.5965313390196564, 'eval_recall': 0.6115, 'eval_runtime': 2.9275, 'eval_samples_per_second': 683.175, 'eval_steps_per_second': 2.733, 'epoch': 1.0}
#015100%|██████████| 8/8 [00:02<00:00,  2.93it/s]#033[A
#033[A
Saving model checkpoint to /opt/ml/model/checkpoint-125
Saving model checkpoint to /opt/ml/model/checkpoint-125
Configuration saved in /opt/ml/model/checkpoint-125/config.json
Configuration saved in /opt/ml/model/checkpoint-125/config.json
Model weights saved in /opt/ml/model/checkpoint-125/pytorch_model.bin
Mo

20%|██        | 250/1250 [02:32<08:23,  1.99it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
***** Running Evaluation *****
  Num examples = 2000
Batch size = 256
Batch size = 256
100%|██████████| 8/8 [00:02<00:00,  3.05it/s]#033[A
#033[A
{'eval_loss': 0.29452672600746155, 'eval_accuracy': 0.9015, 'eval_f1': 0.9028380594099472, 'eval_precision': 0.9081824676999165, 'eval_recall': 0.9015, 'eval_runtime': 2.8064, 'eval_samples_per_second': 712.668, 'eval_steps_per_second': 2.851, 'epoch': 2.0}
#015100%|██████████| 8/8 [00:02<00:00,  3.05it/s]
#033[A
#033[A
Saving model checkpoint to /opt/ml/model/checkpoint-250
Saving model checkpoint to /opt/ml/model/checkpoint-250
Confi

...

70%|███████   | 875/1250 [08:18<03:09,  1.98it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
***** Running Evaluation *****
  Num examples = 2000
Batch size = 256
Batch size = 256
100%|██████████| 8/8 [00:02<00:00,  2.90it/s]#033[A
#033[A
{'eval_loss': 0.15445710718631744, 'eval_accuracy': 0.94, 'eval_f1': 0.9390823010532816, 'eval_precision': 0.9399218107094516, 'eval_recall': 0.94, 'eval_runtime': 2.904, 'eval_samples_per_second': 688.7, 'eval_steps_per_second': 2.755, 'epoch': 7.0}
100%|██████████| 8/8 [00:02<00:00,  2.90it/s]#033[A
#015                                             #033[A
Saving model checkpoint to /opt/ml/model/checkpoint-875
Saving model checkpoint 

/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
80%|████████  | 1000/1250 [09:28<02:03,  2.02it/s]
{'loss': 0.104, 'learning_rate': 1.004e-05, 'epoch': 8.0}
80%|████████  | 1000/1250 [09:28<02:03,  2.02it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
***** Running Evaluation *****
Num examples = 2000
  Batch size = 256
Num examples = 2000
  Batch size = 256
100%|██████████| 8/8 [00:02<00:00,  2.94it/s]#033[A
#033[A
{'eval_loss': 0.14760388433933258, 'eval_accuracy': 0.93

90%|█████████ | 1125/1250 [10:37<01:04,  1.95it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
***** Running Evaluation *****
  Num examples = 2000
Batch size = 256
Batch size = 256
88%|████████▊ | 7/8 [00:02<00:00,  2.73it/s]
#033[A
100%|██████████| 8/8 [00:02<00:00,  2.84it/s]#033[A
#033[A
{'eval_loss': 0.1530248075723648, 'eval_accuracy': 0.9345, 'eval_f1': 0.9345077978012077, 'eval_precision': 0.9346316724320993, 'eval_recall': 0.9345, 'eval_runtime': 2.9642, 'eval_samples_per_second': 674.719, 'eval_steps_per_second': 2.699, 'epoch': 9.0}
#015100%|██████████| 8/8 [00:02<00:00,  2.84it/s]#033[A
#033[A
Saving model checkpoint to /opt/ml/model/checkpoint-1125
Saving mo

100%|██████████| 1250/1250 [11:47<00:00,  1.91it/s]
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
***** Running Evaluation *****
  Num examples = 2000
Batch size = 256
Batch size = 256
100%|██████████| 8/8 [00:02<00:00,  2.92it/s]#033[A
#033[A
{'eval_loss': 0.1613253802061081, 'eval_accuracy': 0.941, 'eval_f1': 0.9413321705151999, 'eval_precision': 0.9419519436781406, 'eval_recall': 0.941, 'eval_runtime': 2.8864, 'eval_samples_per_second': 692.901, 'eval_steps_per_second': 2.772, 'epoch': 10.0}
#015100%|██████████| 8/8 [00:02<00:00,  2.92it/s]#033[A
#033[A
Saving model checkpoint to /opt/ml/model/checkpoint-1250
Saving model checkpoint to /opt/ml/model/checkpoint-1250
Conf


2021-12-03 21:18:52 Uploading - Uploading generated training modelModel weights saved in /opt/ml/model/checkpoint-1250/pytorch_model.bin
Model weights saved in /opt/ml/model/checkpoint-1250/pytorch_model.bin
tokenizer config file saved in /opt/ml/model/checkpoint-1250/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/checkpoint-1250/tokenizer_config.json
Special tokens file saved in /opt/ml/model/checkpoint-1250/special_tokens_map.json
Special tokens file saved in /opt/ml/model/checkpoint-1250/special_tokens_map.json
Deleting older checkpoint [/opt/ml/model/checkpoint-875] due to args.save_total_limit
Deleting older checkpoint [/opt/ml/model/checkpoint-875] due to args.save_total_limit
Deleting older checkpoint [/opt/ml/model/checkpoint-1125] due to args.save_total_limit
Deleting older checkpoint [/opt/ml/model/checkpoint-1125] due to args.save_total_limit
Training completed. Do not forget to share your model on huggingface.co/models =)
Training completed. Do not forg

In [53]:
# adding waiter to see when training is done
waiter = huggingface_estimator.sagemaker_session.sagemaker_client.get_waiter('training_job_completed_or_stopped')
waiter.wait(TrainingJobName=huggingface_estimator.latest_training_job.name)

In [56]:
# from sagemaker import TrainingJobAnalytics

# # Captured metrics can be accessed as a Pandas dataframe
# df = TrainingJobAnalytics(training_job_name=huggingface_estimator.latest_training_job.name)


In [50]:
print(huggingface_estimator.model_data)

s3://emotion-class-197614225699/training_output/emotion-training-2021-12-03-20-58-03-225/output/model.tar.gz


### Save training job name for next session

In [57]:
training_job_name = huggingface_estimator.latest_training_job.name
training_job_name

'emotion-training-2021-12-03-20-58-03-225'

In [58]:
%store training_job_name

Stored 'training_job_name' (str)


In [2]:
#### Upload the fine-tuned model to huggingface.co

In [60]:
import sagemaker.huggingface
import botocore
# from datasets.filesystems import S3FileSystem

In [61]:
import sagemaker
print(sagemaker.__version__)

2.68.0


In [62]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
#sagemaker_session_bucket="samsum-dataset"

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::197614225699:role/bi-sagemaker-access
sagemaker bucket: sagemaker-us-east-1-197614225699
sagemaker session region: us-east-1


In [63]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = 'my_roberta_base_emotion'

os.makedirs(local_path, exist_ok = True)

# download model from S3
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path=local_path, # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

# unzip model
tar = tarfile.open(f"{local_path}/model.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()
os.remove(f"{local_path}/model.tar.gz")

In [26]:
import json
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
import os
import math
import warnings 
from datetime import datetime, date, timedelta

In [28]:
# # read eval and test results 
# with open(f"{local_path}/eval_results.json") as f:
#     eval_results_raw = json.load(f)
#     print(eval_results_raw)
#     df_results = pd.json_normalize(eval_results_raw)